In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# !ls 'gdrive/My Drive/'

In [47]:
"""
  Takes a model and data, modifies the data for the model, and runs the model
"""

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras import backend as K
from numpy import concatenate
from numpy import append
from numpy import array
from numpy import nan

def sign_ae(x, y):
    sign_x = K.sign(x)
    sign_y = K.sign(y)
    delta = x - y
    return sign_x * sign_y * K.abs(delta)


def linex_loss(delta, a=-1, b=1):
    if a!= 0 and b > 0:
        loss = b * (K.exp(a * delta) - a * delta - 1)
        return loss
    else:
        raise ValueError


def linex_loss_val(y_true, y_pred):
    delta = sign_ae(y_true, y_pred)
    res = linex_loss(delta)
    return K.mean(res)


def linex_loss_ret(y_true, y_pred):
    diff_true = y_true[1:] - y_true[:-1]
    diff_pred = y_pred[1:] - y_pred[:-1]

    delta = sign_ae(diff_true, diff_pred)
    res = linex_loss(delta)
    return K.mean(res)

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
      Creates batches of data for the model
    """
    n_vars = 1 if isinstance(data, list) else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def get_data():
    data = pd.read_csv(filepath_or_buffer="https://raw.githubusercontent.com/richychn/market-modeling/master/static/data/levels.csv", index_col="date")
    data = data.apply(pd.to_numeric, errors = "coerce")
    data['spindx'].replace(0, nan, inplace=True)
    data['spindx'].fillna(method='ffill', inplace=True)
    values = data.values
    values = values.astype('float32')
    return values

def data_setup(values, start):
    historic_data = array([])
    for day in values[-10:]:
        historic_data = concatenate((historic_data, day), axis=None)
    historic_data = append(historic_data, historic_data)
    look_back = 10
    time_steps = 10
    reframed = series_to_supervised(values, look_back, time_steps)
    reframed = reframed.append(dict(zip(reframed.columns, historic_data)), ignore_index=True)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(reframed)
    pred_para = array([])
    pred_para = scaled[-1][:-80]
    pred_para = pred_para.reshape(1,10,8)
    return pred_para    

def test_single_investment():
    values = get_data()
    K.clear_session()
    multi_model = load_model("/home/abel/market-modeling/static/models/8-8-LSTM-L.hdf5", custom_objects={'linex_loss_ret': linex_loss_ret})

    investments = []
    earn = 1
    day = 0

    while day < len(values) - 10:
        print(day, len(values - 10))
        pred_para = data_setup(values, day)
#         if not pred_para.shape[0]:
#             break
        yhat = multi_model.predict(pred_para)
        growth = yhat[0][0] - pred_para[0][-1]
        real_growth =  (values[day,:] - values[day-1,:]) /(values[day-1,:])

        best_so_far = 0
        asset = -1
        for num, gr in enumerate(growth):
            if gr > best_so_far:
                best_so_far = gr
                asset = num
        if best_so_far > 0:
            earn = earn * (1 + real_growth[asset])
        investments.append(asset)
        day += 1
    return earn, investments
        

In [48]:
ret = test_single_investment()
print(ret)

0 7931
1 7931
2 7931
3 7931
4 7931
5 7931
6 7931
7 7931
8 7931
9 7931
10 7931
11 7931
12 7931
13 7931
14 7931
15 7931
16 7931
17 7931
18 7931
19 7931
20 7931
21 7931
22 7931
23 7931
24 7931
25 7931
26 7931
27 7931
28 7931
29 7931
30 7931
31 7931
32 7931
33 7931
34 7931
35 7931
36 7931
37 7931
38 7931
39 7931
40 7931
41 7931
42 7931
43 7931
44 7931
45 7931
46 7931
47 7931
48 7931
49 7931
50 7931
51 7931
52 7931
53 7931
54 7931
55 7931
56 7931
57 7931
58 7931
59 7931
60 7931
61 7931
62 7931
63 7931
64 7931
65 7931
66 7931
67 7931
68 7931
69 7931
70 7931
71 7931
72 7931
73 7931
74 7931
75 7931
76 7931
77 7931
78 7931
79 7931
80 7931
81 7931
82 7931
83 7931
84 7931
85 7931
86 7931
87 7931
88 7931
89 7931
90 7931
91 7931
92 7931
93 7931
94 7931
95 7931
96 7931
97 7931
98 7931
99 7931
100 7931
101 7931
102 7931
103 7931
104 7931
105 7931
106 7931
107 7931
108 7931
109 7931
110 7931
111 7931
112 7931
113 7931
114 7931
115 7931
116 7931
117 7931
118 7931
119 7931
120 7931
121 7931
122 7931
123

925 7931
926 7931
927 7931
928 7931
929 7931
930 7931
931 7931
932 7931
933 7931
934 7931
935 7931
936 7931
937 7931
938 7931
939 7931
940 7931
941 7931
942 7931
943 7931
944 7931
945 7931
946 7931
947 7931
948 7931
949 7931
950 7931
951 7931
952 7931
953 7931
954 7931
955 7931
956 7931
957 7931
958 7931
959 7931
960 7931
961 7931
962 7931
963 7931
964 7931
965 7931
966 7931
967 7931
968 7931
969 7931
970 7931
971 7931
972 7931
973 7931
974 7931
975 7931
976 7931
977 7931
978 7931
979 7931
980 7931
981 7931
982 7931
983 7931
984 7931
985 7931
986 7931
987 7931
988 7931
989 7931
990 7931
991 7931
992 7931
993 7931
994 7931
995 7931
996 7931
997 7931
998 7931
999 7931
1000 7931
1001 7931
1002 7931
1003 7931
1004 7931
1005 7931
1006 7931
1007 7931
1008 7931
1009 7931
1010 7931
1011 7931
1012 7931
1013 7931
1014 7931
1015 7931
1016 7931
1017 7931
1018 7931
1019 7931
1020 7931
1021 7931
1022 7931
1023 7931
1024 7931
1025 7931
1026 7931
1027 7931
1028 7931
1029 7931
1030 7931
1031 7931
1032 

1756 7931
1757 7931
1758 7931
1759 7931
1760 7931
1761 7931
1762 7931
1763 7931
1764 7931
1765 7931
1766 7931
1767 7931
1768 7931
1769 7931
1770 7931
1771 7931
1772 7931
1773 7931
1774 7931
1775 7931
1776 7931
1777 7931
1778 7931
1779 7931
1780 7931
1781 7931
1782 7931
1783 7931
1784 7931
1785 7931
1786 7931
1787 7931
1788 7931
1789 7931
1790 7931
1791 7931
1792 7931
1793 7931
1794 7931
1795 7931
1796 7931
1797 7931
1798 7931
1799 7931
1800 7931
1801 7931
1802 7931
1803 7931
1804 7931
1805 7931
1806 7931
1807 7931
1808 7931
1809 7931
1810 7931
1811 7931
1812 7931
1813 7931
1814 7931
1815 7931
1816 7931
1817 7931
1818 7931
1819 7931
1820 7931
1821 7931
1822 7931
1823 7931
1824 7931
1825 7931
1826 7931
1827 7931
1828 7931
1829 7931
1830 7931
1831 7931
1832 7931
1833 7931
1834 7931
1835 7931
1836 7931
1837 7931
1838 7931
1839 7931
1840 7931
1841 7931
1842 7931
1843 7931
1844 7931
1845 7931
1846 7931
1847 7931
1848 7931
1849 7931
1850 7931
1851 7931
1852 7931
1853 7931
1854 7931
1855 7931


2577 7931
2578 7931
2579 7931
2580 7931
2581 7931
2582 7931
2583 7931
2584 7931
2585 7931
2586 7931
2587 7931
2588 7931
2589 7931
2590 7931
2591 7931
2592 7931
2593 7931
2594 7931
2595 7931
2596 7931
2597 7931
2598 7931
2599 7931
2600 7931
2601 7931
2602 7931
2603 7931
2604 7931
2605 7931
2606 7931
2607 7931
2608 7931
2609 7931
2610 7931
2611 7931
2612 7931
2613 7931
2614 7931
2615 7931
2616 7931
2617 7931
2618 7931
2619 7931
2620 7931
2621 7931
2622 7931
2623 7931
2624 7931
2625 7931
2626 7931
2627 7931
2628 7931
2629 7931
2630 7931
2631 7931
2632 7931
2633 7931
2634 7931
2635 7931
2636 7931
2637 7931
2638 7931
2639 7931
2640 7931
2641 7931
2642 7931
2643 7931
2644 7931
2645 7931
2646 7931
2647 7931
2648 7931
2649 7931
2650 7931
2651 7931
2652 7931
2653 7931
2654 7931
2655 7931
2656 7931
2657 7931
2658 7931
2659 7931
2660 7931
2661 7931
2662 7931
2663 7931
2664 7931
2665 7931
2666 7931
2667 7931
2668 7931
2669 7931
2670 7931
2671 7931
2672 7931
2673 7931
2674 7931
2675 7931
2676 7931


3399 7931
3400 7931
3401 7931
3402 7931
3403 7931
3404 7931
3405 7931
3406 7931
3407 7931
3408 7931
3409 7931
3410 7931
3411 7931
3412 7931
3413 7931
3414 7931
3415 7931
3416 7931
3417 7931
3418 7931
3419 7931
3420 7931
3421 7931
3422 7931
3423 7931
3424 7931
3425 7931
3426 7931
3427 7931
3428 7931
3429 7931
3430 7931
3431 7931
3432 7931
3433 7931
3434 7931
3435 7931
3436 7931
3437 7931
3438 7931
3439 7931
3440 7931
3441 7931
3442 7931
3443 7931
3444 7931
3445 7931
3446 7931
3447 7931
3448 7931
3449 7931
3450 7931
3451 7931
3452 7931
3453 7931
3454 7931
3455 7931
3456 7931
3457 7931
3458 7931
3459 7931
3460 7931
3461 7931
3462 7931
3463 7931
3464 7931
3465 7931
3466 7931
3467 7931
3468 7931
3469 7931
3470 7931
3471 7931
3472 7931
3473 7931
3474 7931
3475 7931
3476 7931
3477 7931
3478 7931
3479 7931
3480 7931
3481 7931
3482 7931
3483 7931
3484 7931
3485 7931
3486 7931
3487 7931
3488 7931
3489 7931
3490 7931
3491 7931
3492 7931
3493 7931
3494 7931
3495 7931
3496 7931
3497 7931
3498 7931


4222 7931
4223 7931
4224 7931
4225 7931
4226 7931
4227 7931
4228 7931
4229 7931
4230 7931
4231 7931
4232 7931
4233 7931
4234 7931
4235 7931
4236 7931
4237 7931
4238 7931
4239 7931
4240 7931
4241 7931
4242 7931
4243 7931
4244 7931
4245 7931
4246 7931
4247 7931
4248 7931
4249 7931
4250 7931
4251 7931
4252 7931
4253 7931
4254 7931
4255 7931
4256 7931
4257 7931
4258 7931
4259 7931
4260 7931
4261 7931
4262 7931
4263 7931
4264 7931
4265 7931
4266 7931
4267 7931
4268 7931
4269 7931
4270 7931
4271 7931
4272 7931
4273 7931
4274 7931
4275 7931
4276 7931
4277 7931
4278 7931
4279 7931
4280 7931
4281 7931
4282 7931
4283 7931
4284 7931
4285 7931
4286 7931
4287 7931
4288 7931
4289 7931
4290 7931
4291 7931
4292 7931
4293 7931
4294 7931
4295 7931
4296 7931
4297 7931
4298 7931
4299 7931
4300 7931
4301 7931
4302 7931
4303 7931
4304 7931
4305 7931
4306 7931
4307 7931
4308 7931
4309 7931
4310 7931
4311 7931
4312 7931
4313 7931
4314 7931
4315 7931
4316 7931
4317 7931
4318 7931
4319 7931
4320 7931
4321 7931


5044 7931
5045 7931
5046 7931
5047 7931
5048 7931
5049 7931
5050 7931
5051 7931
5052 7931
5053 7931
5054 7931
5055 7931
5056 7931
5057 7931
5058 7931
5059 7931
5060 7931
5061 7931
5062 7931
5063 7931
5064 7931
5065 7931
5066 7931
5067 7931
5068 7931
5069 7931
5070 7931
5071 7931
5072 7931
5073 7931
5074 7931
5075 7931
5076 7931
5077 7931
5078 7931
5079 7931
5080 7931
5081 7931
5082 7931
5083 7931
5084 7931
5085 7931
5086 7931
5087 7931
5088 7931
5089 7931
5090 7931
5091 7931
5092 7931
5093 7931
5094 7931
5095 7931
5096 7931
5097 7931
5098 7931
5099 7931
5100 7931
5101 7931
5102 7931
5103 7931
5104 7931
5105 7931
5106 7931
5107 7931
5108 7931
5109 7931
5110 7931
5111 7931
5112 7931
5113 7931
5114 7931
5115 7931
5116 7931
5117 7931
5118 7931
5119 7931
5120 7931
5121 7931
5122 7931
5123 7931
5124 7931
5125 7931
5126 7931
5127 7931
5128 7931
5129 7931
5130 7931
5131 7931
5132 7931
5133 7931
5134 7931
5135 7931
5136 7931
5137 7931
5138 7931
5139 7931
5140 7931
5141 7931
5142 7931
5143 7931


5864 7931
5865 7931
5866 7931
5867 7931
5868 7931
5869 7931
5870 7931
5871 7931
5872 7931
5873 7931
5874 7931
5875 7931
5876 7931
5877 7931
5878 7931
5879 7931
5880 7931
5881 7931
5882 7931
5883 7931
5884 7931
5885 7931
5886 7931
5887 7931
5888 7931
5889 7931
5890 7931
5891 7931
5892 7931
5893 7931
5894 7931
5895 7931
5896 7931
5897 7931
5898 7931
5899 7931
5900 7931
5901 7931
5902 7931
5903 7931
5904 7931
5905 7931
5906 7931
5907 7931
5908 7931
5909 7931
5910 7931
5911 7931
5912 7931
5913 7931
5914 7931
5915 7931
5916 7931
5917 7931
5918 7931
5919 7931
5920 7931
5921 7931
5922 7931
5923 7931
5924 7931
5925 7931
5926 7931
5927 7931
5928 7931
5929 7931
5930 7931
5931 7931
5932 7931
5933 7931
5934 7931
5935 7931
5936 7931
5937 7931
5938 7931
5939 7931
5940 7931
5941 7931
5942 7931
5943 7931
5944 7931
5945 7931
5946 7931
5947 7931
5948 7931
5949 7931
5950 7931
5951 7931
5952 7931
5953 7931
5954 7931
5955 7931
5956 7931
5957 7931
5958 7931
5959 7931
5960 7931
5961 7931
5962 7931
5963 7931


6686 7931
6687 7931
6688 7931
6689 7931
6690 7931
6691 7931
6692 7931
6693 7931
6694 7931
6695 7931
6696 7931
6697 7931
6698 7931
6699 7931
6700 7931
6701 7931
6702 7931
6703 7931
6704 7931
6705 7931
6706 7931
6707 7931
6708 7931
6709 7931
6710 7931
6711 7931
6712 7931
6713 7931
6714 7931
6715 7931
6716 7931
6717 7931
6718 7931
6719 7931
6720 7931
6721 7931
6722 7931
6723 7931
6724 7931
6725 7931
6726 7931
6727 7931
6728 7931
6729 7931
6730 7931
6731 7931
6732 7931
6733 7931
6734 7931
6735 7931
6736 7931
6737 7931
6738 7931
6739 7931
6740 7931
6741 7931
6742 7931
6743 7931
6744 7931
6745 7931
6746 7931
6747 7931
6748 7931
6749 7931
6750 7931
6751 7931
6752 7931
6753 7931
6754 7931
6755 7931
6756 7931
6757 7931
6758 7931
6759 7931
6760 7931
6761 7931
6762 7931
6763 7931
6764 7931
6765 7931
6766 7931
6767 7931
6768 7931
6769 7931
6770 7931
6771 7931
6772 7931
6773 7931
6774 7931
6775 7931
6776 7931
6777 7931
6778 7931
6779 7931
6780 7931
6781 7931
6782 7931
6783 7931
6784 7931
6785 7931


7506 7931
7507 7931
7508 7931
7509 7931
7510 7931
7511 7931
7512 7931
7513 7931
7514 7931
7515 7931
7516 7931
7517 7931
7518 7931
7519 7931
7520 7931
7521 7931
7522 7931
7523 7931
7524 7931
7525 7931
7526 7931
7527 7931
7528 7931
7529 7931
7530 7931
7531 7931
7532 7931
7533 7931
7534 7931
7535 7931
7536 7931
7537 7931
7538 7931
7539 7931
7540 7931
7541 7931
7542 7931
7543 7931
7544 7931
7545 7931
7546 7931
7547 7931
7548 7931
7549 7931
7550 7931
7551 7931
7552 7931
7553 7931
7554 7931
7555 7931
7556 7931
7557 7931
7558 7931
7559 7931
7560 7931
7561 7931
7562 7931
7563 7931
7564 7931
7565 7931
7566 7931
7567 7931
7568 7931
7569 7931
7570 7931
7571 7931
7572 7931
7573 7931
7574 7931
7575 7931
7576 7931
7577 7931
7578 7931
7579 7931
7580 7931
7581 7931
7582 7931
7583 7931
7584 7931
7585 7931
7586 7931
7587 7931
7588 7931
7589 7931
7590 7931
7591 7931
7592 7931
7593 7931
7594 7931
7595 7931
7596 7931
7597 7931
7598 7931
7599 7931
7600 7931
7601 7931
7602 7931
7603 7931
7604 7931
7605 7931
